In [1]:
import pandas as pd
import random
import io
import numpy as np
import string
import MeCab 
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.metrics.pairwise import cosine_similarity
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, RepeatVector
from keras.utils import np_utils
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
from six.moves import cPickle
from keras import backend as K
from keras.utils import np_utils

Using TensorFlow backend.
/home/ifte/amiebot_project/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ifte/amiebot_project/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ifte/amiebot_project/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ifte/amiebot_project/lib/python3.6/sit

In [3]:
mecab = MeCab.Tagger('-Owakati')

p = Path("/home/ifte/amiebot_project/amie-HelpBot/mobicontrol_data/corpus_mobicontrol.csv")
df = pd.read_csv(p, header=0, usecols=["page", "text"])

In [4]:
import re

def clean_text(text):
    replaced_text = re.sub(r'[【】]', ' ', text)       # 【】の除去
    replaced_text = re.sub(r'[・_]', '', replaced_text)       # ・ の除去
    replaced_text = re.sub(r'[（）()]', ' ', replaced_text)     # （）の除去
    replaced_text = re.sub(r'[［］\[\]]', ' ', replaced_text)   # ［］の除去
    replaced_text = re.sub(r'[@＠]\w+', '', replaced_text)  # メンションの除去
    replaced_text = re.sub(r'https?:\/\/.*?[\r\n ]', '', replaced_text)  # URLの除去
    replaced_text = re.sub(r'　', ' ', replaced_text)  # 全角空白の除去
    replaced_text = re.sub(r'\d+', '', replaced_text) # 数字の除去
    replaced_text = re.sub(r'[-/。,、.=]', ' ', replaced_text)
    return replaced_text

In [5]:
stop_words_ja = []
STOPWORD_FILE = Path("/home/ifte/amiebot_project/amie-HelpBot/amie_helpbot/" + '/assets/learning/stopword_ja.txt')
with open(STOPWORD_FILE, encoding='utf-8') as fr:
    stop_words_ja = fr.read().splitlines()

In [6]:
y = df[["page"]]
X = df["text"].apply(lambda x: mecab.parse(x).strip("\n"))
df['parsed'] = X 

In [7]:
classes = df.page.unique()
c_size = 100

result = pd.DataFrame()

for i in list(classes):
    arr_index = df[df.page == i].parsed.values
    data = ''.join(list(arr_index))
    
    all_data = clean_text(data)
    
    arr_words = np.array(all_data.split())
    #print(arr_words) 
    
    num = arr_words.shape[0]//c_size
    full = c_size * num
    rest = arr_words.shape[0] - full
    
    pad = np.zeros([c_size-rest], dtype=int)
    sc = np.concatenate((arr_words,pad))
    features = sc.reshape(num+1, c_size)
    
    df_v = pd.DataFrame(features)
    df_v["cls"] = i
    
    result = pd.concat([result, df_v]) 

    
#print(result)
#result.to_csv('processed.csv')


# Splitting into fixed length however neural network works with fixed length input, 
# we need to define this length by getting rid or trimming inputs that are longer than this length
# But we dont want to trim any information hence the above approach will work well
#################################################
#MAX_LEN = 20
#train_data = train_data[train_data.Question.apply(lambda x: len(x.split())) < MAX_LEN]
#################################################
# data is cleaned which we did above
#train_data.Question = train_data.Question.apply(lambda x: (re.sub('[^\u0620-\uFEF0\s]', '', x)).strip())
#################################################
#We have to pad sequences that are shorter than MAX_LEN
#train_data = pad_sequences(train_data, padding='post', truncating='post', maxlen=MAX_LEN)

# Tokenization is the process of reading the text we have and creating a vocabulary based on some parameters, 
# then using this vocabulary we define an index where every word in the vocabulary has its ID.
#################################################
#tokenizer = Tokenizer(num_words=NUM_WORDS, lower=False)
#tokenizer.fit_on_texts(train_data["Question"].values)
#train_data = tokenizer.texts_to_sequences(train_data["Question"].values)

In [8]:
Data_cols = result.iloc[:,:-1]
tog = Data_cols.apply(lambda x: ' '.join(x), axis=1)
#tog = result.iloc[:,:].apply(lambda x: ' '.join(x), axis=1)
sent = list(tog.values)

In [9]:
NUM_WORDS = len(np.unique(Data_cols.values.ravel()))

In [10]:
tokenizer = Tokenizer(num_words=NUM_WORDS, lower=False)
tokenizer.fit_on_texts(tog.values)
train_data = tokenizer.texts_to_sequences(tog.values)

In [11]:
train_data = pad_sequences(train_data, padding='post', truncating='post', maxlen=101, value=0)

In [42]:
len(train_data)
train_data.shape
train_data

array([[ 28,   1,  16, ..., 479,   9,   0],
       [  1,  35,  33, ...,  59,   3,   0],
       [  2,  59, 202, ...,   8, 148,   0],
       ...,
       [311, 312, 298, ..., 212,   4,   0],
       [ 76,   2, 191, ...,  28,  25,   0],
       [  1, 128,   8, ...,   6,   6,   0]], dtype=int32)

In [15]:
#BATCH_SIZE = 32 # Batch size for GPU
#MAX_LEN = 20 # Padding length (# of words)
LSTM_EMBED = 10 # Number of LSTM nodes

In [16]:
model = Sequential()
model.add(Embedding(NUM_WORDS, 100, input_length=101))
model.add(LSTM(LSTM_EMBED, dropout=0.2, recurrent_dropout=0.2, input_shape=(train_data.shape[1], NUM_WORDS)))
model.add(RepeatVector(train_data.shape[-1]))
model.add(LSTM(LSTM_EMBED, dropout=0.2, recurrent_dropout=0.2, return_sequences=True))
model.add(Dense(NUM_WORDS, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')
model.fit(train_data, np.expand_dims(train_data, -1), epochs=3, batch_size=101)

model.save("lstm-encoder.h5")

Epoch 1/3
6873/6873 [==============================] - 177s 26ms/step - loss: 8.3560
Epoch 2/3
6873/6873 [==============================] - 189s 27ms/step - loss: 7.0895
Epoch 3/3
6873/6873 [==============================] - 190s 28ms/step - loss: 6.3625


In [17]:
K.set_learning_phase(False)
#tokenizer = cPickle.load(open("lstm-autoencoder-tokenizer.pickle", "rb"))
# Read the encoder model
#model = load_model("lstm-encoder.h5")

In [18]:
# Create the encoding function
encode = K.function([model.input, K.learning_phase()], [model.layers[1].output])

In [19]:
Questions = np.squeeze(np.array(encode([train_data])))

In [68]:
question = input('Please enter a question: \n')
#question = stemmer.stem(question)
question = tokenizer.texts_to_sequences([question])
question = pad_sequences(question, padding='post', truncating='post', maxlen=101)
question = np.squeeze(encode([question]))

rank = cosine_similarity(question.reshape(1, -1), Questions)

   

Please enter a question: 
IP connections


In [69]:
ranking = -np.sort(-rank)
print(ranking)
t = np.argsort(ranking, axis=1)
top = t.reshape(-1,1)
top10 = top[0:10]

[[1.0000001 1.0000001 1.0000001 ... 0.9999981 0.9999975 0.9999973]]


In [70]:
top10[0:10]
#print(np.sort(rank))
# Sorting descendent order
#top

array([[6872],
       [6871],
       [6870],
       [6869],
       [6868],
       [6867],
       [6866],
       [6865],
       [6864],
       [6863]])

In [74]:
top10=np.argsort(rank, axis=-1).T[-5:]
top10

array([[6594],
       [2086],
       [1494],
       [2762],
       [4514]])

In [76]:
np.argsort(rank, axis=-1).T[-10:]

array([[1992],
       [3576],
       [5947],
       [ 660],
       [2151],
       [6594],
       [2086],
       [1494],
       [2762],
       [4514]])

In [77]:
for item in range(len(top10)):
    #print(data['Answer'].iloc[item].values[0])
    idx = top[item,0]
    
        # if the dataset file structure "Quesition | Answer" then use following line  
        #print(data['Answer'].iloc[item].values[0])
    print("PageID: ",result.iloc[idx,-1],"idx",idx,"=",tog.values[idx])
    print("\n ########## \n")

PageID:  330 idx 6872 = の URL が 表示 さ れ ます SSL Cert MobiControl サーバ と の SSL 通信 の ため の 証明 書 の ファイル 名 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

 ########## 

PageID:  330 idx 6871 = チェック を 入れ ます エージェント の 挙動 を ログ に 取り 後 に SOTI に 送り 問題 分析 を し て 貰い ます Connect Disconnect PC から MobiControl サーバ へ の 接続 または 切断 の アクション を 執る とき に この ボタン を 押し ます Status PC の 状況 を 表示 し ます Profile PC に 設定 さ れ て いる 構成 プロファイル の 名前 を 表示 し ます Events PC と MobiControl サーバ と の 交信 記録 Packages パッケージ スタジオ で 作成 し た アプリ の パッケージ 名 が 表示 さ れ ます Servers 接続 可能 な MobiControl サーバ

 ########## 

PageID:  330 idx 6870 = Co Ltd Tokyo タブ 説明 General Device Name MobiControl コンソール で の この PC で の 名前 です ここ で 名前 変更 を する こと が でき ます Allow Inboud TCP IP Connections これ に チェック を 入れる と PC が MobiControl サーバ に 接続 し て い なく て も コンソール 側 から PC に 接続 し て リモート 操作 を し て 貰え ます 但し コンソール と PC と の 間 に Firewall が ある と 接続 でき ませ ん Log to file Normally Off エージェ